<a href="https://colab.research.google.com/github/AgeCoder/Deep-learning-and-llms/blob/main/Sentiment_Analysis_on_IMDB_Reviews_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from pathlib import Path

#Tensorflow & Keras related packages
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [2]:
import tensorflow_datasets as tfds

In [3]:
train, val, test = tfds.load(name="imdb_reviews",split=["train[:80%]", "train[80%:]", "test"],
                             as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.HTNIHW_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.HTNIHW_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.HTNIHW_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
print (len(train))
print (len(val))
print (len(test))

20000
5000
25000


In [6]:
for element in train.take(2):
    print (element)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [7]:
for review,label in train.take(1):
    print (review)
    print (label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [8]:
train_data = train.batch(32)
val_data = val.batch(32)
test_data = test.batch(32)

In [9]:
for reviews, labels in train_data.take(1):
    print ("Reviews shape", reviews.shape)
    print ("Labels shape", labels.shape, "\n")

    print ('First Review:', reviews[0].numpy().decode("utf-8"), "\n")
    print ('First Label:', labels[0].numpy())

Reviews shape (32,)
Labels shape (32,) 

First Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it. 

First Label: 0


In [10]:
train_data_onlyreviews = train_data.map(lambda x,y : x)

In [11]:
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization

max_tokens = 10000
text_vectorization = TextVectorization(
                                        max_tokens=max_tokens,
                                        output_mode = "multi_hot"
)

In [12]:
text_vectorization.adapt(train_data_onlyreviews)

In [13]:
vocab = text_vectorization.get_vocabulary()
vocab

['[UNK]',
 np.str_('the'),
 np.str_('and'),
 np.str_('a'),
 np.str_('of'),
 np.str_('to'),
 np.str_('is'),
 np.str_('in'),
 np.str_('it'),
 np.str_('i'),
 np.str_('this'),
 np.str_('that'),
 np.str_('br'),
 np.str_('was'),
 np.str_('as'),
 np.str_('for'),
 np.str_('with'),
 np.str_('movie'),
 np.str_('but'),
 np.str_('film'),
 np.str_('on'),
 np.str_('not'),
 np.str_('you'),
 np.str_('his'),
 np.str_('are'),
 np.str_('have'),
 np.str_('be'),
 np.str_('he'),
 np.str_('one'),
 np.str_('its'),
 np.str_('at'),
 np.str_('all'),
 np.str_('by'),
 np.str_('an'),
 np.str_('they'),
 np.str_('who'),
 np.str_('from'),
 np.str_('so'),
 np.str_('like'),
 np.str_('her'),
 np.str_('or'),
 np.str_('just'),
 np.str_('about'),
 np.str_('if'),
 np.str_('has'),
 np.str_('out'),
 np.str_('some'),
 np.str_('there'),
 np.str_('what'),
 np.str_('good'),
 np.str_('very'),
 np.str_('when'),
 np.str_('more'),
 np.str_('she'),
 np.str_('my'),
 np.str_('even'),
 np.str_('would'),
 np.str_('up'),
 np.str_('no'),
 np

In [14]:
train_multihot_ugram = train_data.map(lambda x,y: (text_vectorization(x),y))
val_multihot_ugram = val_data.map(lambda x,y: (text_vectorization(x),y))
test_multihot_ugram = test_data.map(lambda x,y: (text_vectorization(x),y))

In [15]:
type(train_multihot_ugram)

tensorflow.python.data.ops.map_op._MapDataset

In [16]:
for reviews, labels in train_multihot_ugram.take(1):
    print ("Reviews shape", reviews.shape)
    print ("Labels shape", labels.shape, "\n")

    print ('First Review:', reviews[0], "\n")
    print ('First Label:', labels[0].numpy())

Reviews shape (32, 10000)
Labels shape (32,) 

First Review: tf.Tensor([1 1 1 ... 0 0 0], shape=(10000,), dtype=int64) 

First Label: 0


In [17]:
for reviews, labels in train_multihot_ugram.take(1):
    # Convert tensor to numpy for easy to view
    reviews_np = reviews.numpy()

    # Create DataFrame
    df = pd.DataFrame(reviews_np, columns=vocab)
    break
df

,[UNK],the,and,a,of,to,is,in,it,i,...,eliminated,effortbr,edits,dug,duff,drifter,dreadfully,dreaded,download,dock
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [19]:
hidden_units = 16

def model_ugram_dense():

    # Define Input shape
    inputs = keras.Input(shape = (max_tokens,))

    # Dense Hidden Layer
    x = layers.Dense(hidden_units, activation="relu")(inputs)

    # Dropout Layer
    x = layers.Dropout(0.5)(x)

    # Dense output Layer
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [20]:
model = model_ugram_dense()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       160,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,033 (625.13 KB)

 Trainable params: 160,033 (625.13 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model = model_ugram_dense()
path = Path("./models/model_ugram_dense.keras")

In [22]:
def train_evaluate(model,path,train,val,test):

    # Call backs
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = path,
                                                       save_best_only=True) # Save only best model

    earlystop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                                 restore_best_weights=True)
    callbacks = [checkpoint_cb,earlystop_cb]

    # Compile the model
    model.compile(optimizer="rmsprop", loss='binary_crossentropy',  metrics = ["accuracy"])

    # Train the model
    history = model.fit(train, validation_data = val, callbacks=callbacks, epochs=20)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test)

    return (history,test_accuracy)

In [23]:
(history_ugram_dense,test_accuracy_ugram_dense) = train_evaluate(model,path,
                                                                 train_multihot_ugram,
                                                                 val_multihot_ugram,
                                                                 test_multihot_ugram)

Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.7612 - loss: 0.4930 - val_accuracy: 0.8900 - val_loss: 0.2860
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.8908 - loss: 0.2899 - val_accuracy: 0.8922 - val_loss: 0.2830
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9093 - loss: 0.2490 - val_accuracy: 0.8914 - val_loss: 0.2938
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9156 - loss: 0.2426 - val_accuracy: 0.8896 - val_loss: 0.3084
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9224 - loss: 0.2334 - val_accuracy: 0.8876 - val_loss: 0.3248
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9270 - loss: 0.2257 - val_accuracy: 0.8852 - val_loss: 0.3422
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9301 - loss: 0.2230 - val_accuracy: 0.8854 - val_loss: 0.3416
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8846 - loss: 0.2948


In [24]:
print (f"Accuracy on the test data set is {test_accuracy_ugram_dense}")

Accuracy on the test data set is 0.8860800266265869


In [25]:
test_probabilities = model.predict(test_multihot_ugram)
test_probabilities

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step


array([[9.1622835e-01],
       [9.9063170e-01],
       [9.4574434e-04],
       ...,
       [8.3881867e-04],
       [9.3759233e-01],
       [9.4254202e-01]], dtype=float32)

In [26]:
(test_probabilities >= 0.5).astype(int)

array([[1],
       [1],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [27]:
for review, label in test_multihot_ugram.take(1):
    print (label)

tf.Tensor([1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 1], shape=(32,), dtype=int64)


In [28]:
max_tokens = 10000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode = "tf_idf"
)
# Vectorize the reviews
# apply text vectorization on training data reviews to index the vocabulary
text_vectorization.adapt(train_data_onlyreviews)

vocab = text_vectorization.get_vocabulary()

# reviews are converted into multihot
train_tfidf = train_data.map(lambda x,y: (text_vectorization(x),y))
val_tfidf = val_data.map(lambda x,y: (text_vectorization(x),y))
test_tfidf= test_data.map(lambda x,y: (text_vectorization(x),y))

In [29]:
for reviews, labels in train_tfidf.take(1):

    print ('First Review:', reviews[0], "\n")
    print ('First Label:', labels[0].numpy())

First Review: tf.Tensor([44.46671    1.3945957  0.7109349 ...  0.         0.         0.       ], shape=(10000,), dtype=float32) 

First Label: 0


In [30]:
for reviews, labels in train_tfidf.take(1):
    # Convert tensor to numpy for easy to view
    reviews_np = reviews.numpy()

    # Create DataFrame
    df = pd.DataFrame(reviews_np, columns=vocab)
    break
df

,[UNK],the,and,a,of,to,is,in,it,i,...,eliminated,effortbr,edits,dug,duff,drifter,dreadfully,dreaded,download,dock
0,44.466709,1.394596,0.710935,1.420396,0.719501,0.000000,0.749473,2.278677,0.779585,1.670063,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.116677,3.486489,2.843740,1.420396,3.597504,2.901548,0.749473,0.000000,0.779585,5.010190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,72.258400,6.972979,2.843740,3.550989,2.878003,3.626935,0.000000,1.519118,0.779585,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27.791693,3.486489,2.843740,3.550989,2.158502,0.725387,0.749473,0.759559,0.779585,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22.233355,4.881085,0.710935,1.420396,1.439001,1.450774,0.000000,0.759559,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,122.283447,7.670277,5.687479,6.391780,8.634008,1.450774,7.494730,6.076470,2.338756,0.835032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,350.175323,16.735149,8.531219,5.681582,10.073010,7.253869,3.747365,5.316912,1.559171,4.175158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,72.258400,4.881085,2.843740,2.840791,4.317004,1.450774,2.248419,0.000000,1.559171,0.835032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,44.466709,10.459468,5.687479,2.840791,3.597504,5.803095,2.997892,4.557353,3.897927,3.340126,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,33.350033,2.789191,0.710935,3.550989,3.597504,2.176161,4.496838,0.000000,2.338756,0.835032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [35]:
hidden_units = 16

def model_tfidf_dense():
    inputs = keras.Input(shape = (max_tokens,)) # Define Input shape

    x = layers.Dense(hidden_units, activation="relu")(inputs) # Dense Hidden Layer

    x = layers.Dropout(0.5)(x) # Dropout Layer

    outputs = layers.Dense(1, activation="sigmoid")(x) # Dense output Layer

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [36]:
model = model_tfidf_dense()
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 10000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │       160,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,033 (625.13 KB)

 Trainable params: 160,033 (625.13 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
(history_tfidf_dense,test_accuracy_tfidf_dense) = train_evaluate(model,path,
                                                                 train_tfidf,
                                                                 val_tfidf,
                                                                 test_tfidf)

Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7423 - loss: 0.5613 - val_accuracy: 0.8854 - val_loss: 0.2968
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8838 - loss: 0.3028 - val_accuracy: 0.8906 - val_loss: 0.3141
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9005 - loss: 0.2552 - val_accuracy: 0.8838 - val_loss: 0.3205
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9096 - loss: 0.2444 - val_accuracy: 0.8898 - val_loss: 0.3360
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9185 - loss: 0.2238 - val_accuracy: 0.8920 - val_loss: 0.3403
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9234 - loss: 0.2248 - val_accuracy: 0.8896 - val_loss: 0.3589
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8788 - loss: 0.3146


In [38]:
print (f"Accuracy on the test data set is {test_accuracy_tfidf_dense}")



Accuracy on the test data set is 0.8795199990272522


In [4]:
!pip install nbstripout
!nbstripout --install

fatal: --local can only be used inside a git repository
Installation failed: not a git repository!
